# DuckDB Python Quickstart – Part 1

This notebook demonstrates the foundational features of the DuckDB Python API. DuckDB is an in-process analytical database that allows you to run fast SQL queries directly on data in files or memory without a separate server.

In this part, we cover:

*   Installation and connecting
*   Basic querying using `sql()` and `execute()`
*   Ingesting data from files (CSV, Parquet)
*   Using the Relational API for programmatic query building

## Getting Started: Installation and Connection

Install the `duckdb` Python package.

In [1]:
# Install DuckDB
!pip install duckdb pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import duckdb
print(duckdb.__version__)

1.3.0


DuckDB can connect to an in-memory database (default) or a persistent file.

In [3]:
# Connect to an in-memory database (data lost when connection closes or script ends)
con_memory = duckdb.connect(database=':memory:')

# Connect to or create a persistent database file
# con_file = duckdb.connect(database='my_local_data.duckdb') # Uncomment to use a file

The `duckdb` module also provides a default in-memory connection accessible via `duckdb.sql()`.

## Your First Queries: `sql()` and `execute()`

Use `duckdb.sql()` for quick queries with the default connection.

In [4]:
result = duckdb.sql("SELECT 42 AS answer")

The result is a `DuckDBPyRelation`, which represents a query but isn't materialized yet.

In [5]:
print(type(result))

<class 'duckdb.duckdb.DuckDBPyRelation'>


Materialize the result using methods like `.show()`, `.fetchall()`, `.df()`, etc.

In [6]:
duckdb.sql("SELECT 42 AS answer").show()

┌────────┐
│ answer │
│ int32  │
├────────┤
│     42 │
└────────┘



In [7]:
result_list = duckdb.sql("SELECT 42 AS answer").fetchall()
print(result_list)

[(42,)]


Use `con.execute()` for a DB-API style interface, returning a cursor-like object.

In [8]:
con = duckdb.connect(database=':memory:')
result_cursor = con.execute("SELECT 'hello' || ' ' || 'world' AS greeting")
print(result_cursor.fetchone())

con.close()

('hello world',)


Parameterized queries are best handled with `con.execute()` to prevent SQL injection.

In [9]:
con = duckdb.connect(database=':memory:')
con.execute("CREATE TABLE items (name VARCHAR, value INTEGER)")
con.execute("INSERT INTO items VALUES ('apple', 1), ('banana', 2), ('cherry', 3)")

# Parameterized query using '$param_name' syntax and a dictionary
item_name = 'banana'
result = con.execute("SELECT value FROM items WHERE name = $item_name", {'item_name': item_name})
print(f"Value for {item_name}: {result.fetchone()}")

# Parameterized query using '?' syntax and a tuple/list
item_value = 1
result = con.execute("SELECT name FROM items WHERE value = ?", [item_value])
print(f"Name for value {item_value}: {result.fetchone()}")

con.close()

Value for banana: (2,)
Name for value 1: ('apple',)


## Ingesting Data: From Files to Relations

DuckDB can query files directly using functions like `read_csv`, `read_parquet`, etc. You can call these via Python or within SQL.

In [10]:
con = duckdb.connect(database=':memory:') # Use a new in-memory connection

# Install and load the httpfs extension to read from URLs
con.sql("INSTALL httpfs")
con.sql("LOAD httpfs")

# Read a CSV from a URL into a relation
population_relation = con.read_csv("https://bit.ly/3KoiZR0")

print(type(population_relation))

<class 'duckdb.duckdb.DuckDBPyRelation'>


Query the relation using SQL.

In [11]:
con.sql("SELECT Country, Population FROM population_relation LIMIT 5").show()

┌─────────────────┬────────────┐
│     Country     │ Population │
│     varchar     │   int64    │
├─────────────────┼────────────┤
│ Afghanistan     │   31056997 │
│ Albania         │    3581655 │
│ Algeria         │   32930091 │
│ American Samoa  │      57794 │
│ Andorra         │      71201 │
└─────────────────┴────────────┘



Persist the data into a DuckDB table using `.to_table()` for better performance on repeated access.

In [12]:
# The 'population_relation' object is still available
population_relation.to_table("population") # Creates a table named 'population'

# Now query the persistent table
con.sql("SELECT COUNT(*) FROM population").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          227 │
└──────────────┘



## The Relational API: Building Queries Programmatically

`DuckDBPyRelation` objects are query builders. Chain methods like `filter`, `project`, `limit`, `order`, `aggregate`, etc., to construct queries programmatically. Execution is lazy until a materializing method is called.

In [13]:
# Use the population_relation created earlier
(population_relation
 .filter("Population > 10000000")
 .project("Country, Population")
 .limit(5)
 .show()
)

┌──────────────┬────────────┐
│   Country    │ Population │
│   varchar    │   int64    │
├──────────────┼────────────┤
│ Afghanistan  │   31056997 │
│ Algeria      │   32930091 │
│ Angola       │   12127071 │
│ Argentina    │   39921833 │
│ Australia    │   20264082 │
└──────────────┴────────────┘



You can save intermediate results as variables.

In [14]:
over_10m_population = population_relation.filter("Population > 10000000")

# Aggregate on the filtered relation
(over_10m_population
 .aggregate("""
    Region,
    count(*) AS country_count,
    CAST(avg(Population) AS int) AS avg_pop
    """)
 .order("avg_pop DESC")
 .show()
)

┌─────────────────────────────────────┬───────────────┬───────────┐
│               Region                │ country_count │  avg_pop  │
│               varchar               │     int64     │   int32   │
├─────────────────────────────────────┼───────────────┼───────────┤
│ ASIA (EX. NEAR EAST)                │            19 │ 192779730 │
│ NORTHERN AMERICA                    │             2 │ 165771574 │
│ LATIN AMER. & CARIB                 │            10 │  48643375 │
│ C.W. OF IND. STATES                 │             5 │  48487549 │
│ WESTERN EUROPE                      │             9 │  38955933 │
│ NORTHERN AFRICA                     │             4 │  38808343 │
│ NEAR EAST                           │             5 │  32910924 │
│ SUB-SAHARAN AFRICA                  │            21 │  30941436 │
│ EASTERN EUROPE                      │             3 │  23691959 │
│ OCEANIA                             │             1 │  20264082 │
├─────────────────────────────────────┴─────────

The relational API supports set operations (`except_`, `intersect`, `union`) and joins.

In [15]:
# Find countries with population UNDER 10 million using except_
under_10m_population = population_relation.except_(over_10m_population)

(under_10m_population
 .aggregate("""
    Region,
    count(*) AS country_count,
    CAST(avg(Population) AS int) AS avg_pop
    """)
 .order("avg_pop DESC")
 .show()
)

┌─────────────────────────────────────┬───────────────┬─────────┐
│               Region                │ country_count │ avg_pop │
│               varchar               │     int64     │  int32  │
├─────────────────────────────────────┼───────────────┼─────────┤
│ EASTERN EUROPE                      │             9 │ 5426538 │
│ C.W. OF IND. STATES                 │             7 │ 5377686 │
│ SUB-SAHARAN AFRICA                  │            30 │ 3322228 │
│ NORTHERN AFRICA                     │             2 │ 3086881 │
│ ASIA (EX. NEAR EAST)                │             9 │ 2796374 │
│ NEAR EAST                           │            11 │ 2773978 │
│ WESTERN EUROPE                      │            19 │ 2407190 │
│ BALTICS                             │             3 │ 2394991 │
│ LATIN AMER. & CARIB                 │            35 │ 2154024 │
│ OCEANIA                             │            20 │  643379 │
│ NORTHERN AMERICA                    │             3 │   43053 │
├─────────

In [16]:
# Find countries in Eastern Europe using filter
eastern_europe = population_relation.filter("Region ~ '.*EASTERN EUROPE.*'") # Using SIMILAR TO shorthand

# Find the intersection of Eastern Europe and over 10m population
(eastern_europe
 .intersect(over_10m_population)
 .project("Country, Population")
 .show()
)

┌─────────────────┬────────────┐
│     Country     │ Population │
│     varchar     │   int64    │
├─────────────────┼────────────┤
│ Czech Republic  │   10235455 │
│ Romania         │   22303552 │
│ Poland          │   38536869 │
└─────────────────┴────────────┘



Joining relations. We'll create a small relation from Pandas for this.

In [18]:
import pandas as pd

# con = duckdb.connect()           # or reuse your existing `con`

# your DataFrame
region_nicknames_df = pd.DataFrame(
    {
        "Region": [
            "ASIA (EX. NEAR EAST)     ",
            "NORTHERN AMERICA       ",
            "LATIN AMER. & CARIB      ",
            "C.W. OF IND. STATES      ",
            "WESTERN EUROPE           ",
        ],
        "Nickname": [
            "Asian Tigers",
            "North America",
            "Latin America",
            "CIS States",
            "Western Europe",
        ],
    }
)

# convert the DataFrame to a DuckDB relation
region_nicknames_rel = con.from_df(region_nicknames_df)
#            └────┬────┘
#                  • this is the right helper

# build the rest of the pipeline exactly as you had it
population_trimmed = population_relation.project(
    "Country, Population, trim(Region) AS Region"
)
nicknames_trimmed = region_nicknames_rel.project(
    "trim(Region) AS Region, Nickname"
)

(population_trimmed
 .join(nicknames_trimmed, "Region")
 .project("Country, Population, Nickname")
 .limit(5)
 .show())


┌────────────────────┬────────────┬────────────────┐
│      Country       │ Population │    Nickname    │
│      varchar       │   int64    │    varchar     │
├────────────────────┼────────────┼────────────────┤
│ Afghanistan        │   31056997 │ Asian Tigers   │
│ Andorra            │      71201 │ Western Europe │
│ Anguilla           │      13477 │ Latin America  │
│ Antigua & Barbuda  │      69108 │ Latin America  │
│ Argentina          │   39921833 │ Latin America  │
└────────────────────┴────────────┴────────────────┘



You can also reference relation objects directly within SQL queries executed via `con.sql()`.

In [26]:
con.sql("""
-- Query the 'over_10m_population' relation object directly in SQL
SELECT Country, "GDP ($ per capita)"
FROM over_10m_population
WHERE "GDP ($ per capita)" > 29000
LIMIT 5
""").show()

┌────────────────┬────────────────────┐
│    Country     │ GDP ($ per capita) │
│    varchar     │       int64        │
├────────────────┼────────────────────┤
│ Belgium        │              29100 │
│ Canada         │              29800 │
│ United States  │              37800 │
└────────────────┴────────────────────┘



## Wrapping Up

This covers the basics: installation, connections, querying with `sql()` and `execute()`, direct file ingestion, and building queries with the Relational API.

These features provide a powerful way to handle data analysis directly within Python.

In Part 2, we'll explore integrations with Pandas, Polars, and Arrow, and using Python UDFs.

In [ ]:
# Close the connection when done
con.close()